<a href="https://colab.research.google.com/github/TheProffesional/Formacion-Bosonit/blob/main/KAFKA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En esta guía daremos nuestros primeros pasos con Presto para realizar consultas sobre
topics de kafka. Por ende lo primero que deberemos hacer es arrancar todos los
procesos involucrados.
Así que lo primero que haremos es crear 4 consolas (por ejemplo crear 1 consola y luego
añadir 3 pestañas)
En la primera lanzaremos zookeeper



```
cd kafka_2.11-2.1.0/
zookeeper-server-start.sh config/zookeeper.properties
```

En la segunda una vez arrancado zookeeper ejecutamos kafka
En caso de que no nos encontremos en la carpeta de instalación de kafka nos desplazamos hacia ella



```
cd kafka_2.11-2.1.0/
kafka-server-start.sh config/server.properties
```
En una tercera consola vamos a crear un nuevo topic que llamaremos “text_topic"



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic text_topic --
create --partitions 3 --replication-factor 1
```

Luego en esta misma consola crearemos un producer y comenzaremos a insertar 
elementos en el topic recién creado
Al finalizar la entrada de texto NO cerraremos la consola



```
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic
text_topic
```


```
>Hola mundo
>Hola mundo de nuevo
>Tecrera frase de texto
>Cuarta y últim
>
```





Luego en una cuarta consola vamos a empezar a configurar a Presto



```
cd $HOME
cd presto-server-0.213/
```
Crearemos el fichero kafka.properties ya sea utilizando vi o nano, de esta forma crearemos una nueva fuente de datos.

En este fichero indicamos los nombres de tablas, el nombre de conexión y los nodos de kafka (en nuestro caso nuestro cluster consta de un único nodo) y si han de ocultarse las columnas 
internas



```
vi etc/catalog/kafka.properties
```

Luego arrancamos el servidor de presto




```
bin/launcher start
```

En otra consola lanzamos el cliente de consulta de linea de comandos de presto



```
cd $HOME
./presto --catalog kafka --schema default
```



Lista las tablas existente y veremos la primera recién creada



```
presto:default> SHOW TABLES;
presto:default> DESCRIBE text_topic;
```

Al hacer el DESCRIBE únicamente se listan las columnas internas (relacionadas con kafka)

Consultamos el topic y vemos los mensajes




```
presto:default> SELECT _message FROM text_topic;

```

Una vez hemos comprobado que se ha listado todo el contenido del topic 
Desde la consola donde tenemos el producer introduciremos más texto y posteriormente desde la otra consola donde tenemos el cliente de consulta de presto 

volveremos a consultar el topic



```
presto:default> SELECT _message FROM text_topic;
```

Ya hemos terminado la primera parte de este bloque, ahora procederemos a crear un nuevo topic llamado csv_topic cuyos datos serán transmitidos en formato csv
Cancelamos el producer (Ctrl + C) que habíamos activado para insertar elementos al text_topic
Para crear un nuevo producer para insertar elementos al nuevo topic csv_topic y los consultaremos usando presto como si fuera una tabla



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic csv_topic --
create --partitions 3 --replication-factor 1
```

Arrancamos de nuevo el producer pero para insertar elementos al topic recién creado, es decir, csv_topic



```
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic
csv_topic
```

Añadimos un elemento al topic. Hay que tener muchísimo cuidado con las comillas



```
>"JOSE ANTONIO","38","75.5","C/ LIMONERO 39, 1-C"
```

